In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns
from itertools import product
from itertools import combinations_with_replacement
import torch
import time
from torch.autograd import Variable
%matplotlib inline

In [9]:
from utils import *
# %load_ext autoreload
# %autoreload 2

In [32]:
# adult = pd.read_csv('../labels.txt', delimiter='\t', header=0, names=['user_id','website','rating'])
trec = pd.read_csv('../trec-rf10-crowd/trec-rf10-data.txt', delimiter='\t')

r = pd.Series([2,3,2,3], index=[1,2,0,-2])
trec['label_bin'] = trec['label'].map(r)
trec.head(10)

,topicID,workerID,docID,gold,label,label_bin
0,20002,w1,clueweb09-en0000-66-24091,-1,0,2
1,20002,w1,clueweb09-en0001-31-15410,-1,0,2
2,20002,w1,clueweb09-en0000-05-22942,-1,0,2
3,20002,w1,clueweb09-en0000-05-22943,-1,0,2
4,20002,w1,clueweb09-en0006-85-33191,2,2,3
5,20002,w2,clueweb09-en0000-66-24091,-1,1,2
6,20002,w2,clueweb09-en0001-31-15410,-1,1,2
7,20002,w2,clueweb09-en0000-05-22942,-1,0,2
8,20002,w2,clueweb09-en0000-05-22943,-1,0,2
9,20002,w2,clueweb09-en0006-85-33191,2,-2,3


In [33]:
# testframe = create_user_task_ids(adult, 'user_id', 'website', 'rating')
testframe = create_user_task_ids(trec, 'workerID', 'docID', 'label_bin')
testframe.head()

,topicID,workerID,docID,gold,label,label_bin,task_id,uid,bin
0,20002,w1,clueweb09-en0000-66-24091,-1,0,2,0,0,2
1,20002,w1,clueweb09-en0001-31-15410,-1,0,2,1,0,2
2,20002,w1,clueweb09-en0000-05-22942,-1,0,2,2,0,2
3,20002,w1,clueweb09-en0000-05-22943,-1,0,2,3,0,2
4,20002,w1,clueweb09-en0006-85-33191,2,2,3,4,0,3


In [35]:
completed, values, ind = compute_individual_dist(testframe)

In [71]:
start = time.time()
for user_index in range(values.shape[0]):
    #check for full joint distribution or add a prior later
    if np.sum(ind[user_index]==0) > 0:
        continue
    #compute delta matrices with all other users where applicable
    else:
        #create a mask so that other half of tasks can be used later to find score matrix
        mask = np.random.randint(0,2,values.shape).astype(bool)
        delta_matrices, t_m_i_1, _ = compute_deltas(0, completed, values, ind, mask, False)
        score_matrices, t_m_i_2 = compute_deltas(0, completed, values, ind, ~mask, True)
        print(np.sum(t_m_i_1), np.sum(t_m_i_2))
        print(len(np.intersect1d(np.array(t_m_i_1), np.array(t_m_i_2))))
        print(regret(score_matrices, delta_matrices, np.intersect1d(np.array(t_m_i_1), np.array(t_m_i_2))))
print(time.time()-start)

/Users/annahilgard/Documents/CS281/final_project/cs281-project/Dataverse Files/utils.py:55: RuntimeWarning: divide by zero encountered in true_divide
  joint = joint/joint[:,1:].sum(axis=1)[:,None]
/Users/annahilgard/Documents/CS281/final_project/cs281-project/Dataverse Files/utils.py:55: RuntimeWarning: invalid value encountered in true_divide
  joint = joint/joint[:,1:].sum(axis=1)[:,None]


31 34
2
-0.0138148033172
23 32
2
-2.22044604925e-16
29 29
2
-0.153665873326
25 37
2
-0.0314271991021
26 27
2
0.0220210532424
29 27
2
-1.11022302463e-16
30 31
2
-0.00482110314254
31 38
2
0.101194996454
31 27
2
0.0540374294463
28 27
2
0.135571072106
27 33
2
0.00855822097148
25 29
2
0.0576504722746
35 25
2
-0.0588325072603
26 28
2
-0.0213565335773
24 30
2
-0.0247762219635


KeyboardInterrupt: 

In [68]:
start = time.time()
for i in range(5):
    print(time.time() - start)
    print(regret(score_matrices, delta_matrices, t_m_i_1))

4.696846008300781e-05
0.218351527534
10.426027059555054
0.211616373914
19.596651077270508
0.218492334803
29.22946786880493
0.218464728912
39.08114194869995
0.218298026568


In [59]:
expected_score(score_matrices[2], delta_matrices[2], np.array(range(2)), np.array(range(2)))

0.47810218978102192

In [65]:
np.sum(np.sum(score_matrices[t_m_i_1], (1,2))==0)

0

In [66]:
score_matrices[t_m_i_1]

array([[[  1.,   1.],
        [  1.,   1.]],

       [[  1.,   1.],
        [  1.,   1.]],

       [[  1.,   1.],
        [  1.,   0.]],

       [[  1.,   0.],
        [  0.,   0.]],

       [[  1.,   1.],
        [  1.,   1.]],

       [[  0.,   1.],
        [  1.,   1.]],

       [[  1.,   0.],
        [  0.,   1.]],

       [[  1.,   1.],
        [  1.,   0.]],

       [[  1.,   1.],
        [  0.,   1.]],

       [[  1.,   1.],
        [  1.,   1.]],

       [[  1.,   1.],
        [  1.,   0.]],

       [[  1.,   1.],
        [  0.,   1.]],

       [[  1.,   0.],
        [  0.,   1.]],

       [[  1.,   1.],
        [  1.,   1.]],

       [[  0.,   1.],
        [  1.,   1.]],

       [[  0.,   1.],
        [  1.,   1.]],

       [[  1.,   1.],
        [  0.,   1.]],

       [[  1.,   1.],
        [  1.,   1.]],

       [[  1.,   0.],
        [  1.,   1.]],

       [[  1.,   0.],
        [  0.,   1.]],

       [[  1.,   1.],
        [  1.,   0.]],

       [[  0.,   1.],
        [  1